In [1]:
import torch # type: ignore
import torch.nn as nn  # type: ignore
import torch.nn.functional as F  # type: ignore
import torch.optim as optim  # type: ignore
from torchvision import datasets, transforms # type: ignore

In [2]:
# Normalize the train and testing data
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

In [4]:
# Load MNIST Dataset
train_dataset = datasets.MNIST(root = './data', train = True, download = True, transform = transform)
test_dataset = datasets.MNIST(root = './data', train = False, download = True, transform = transform)

In [5]:
# Define data loaders
train_loader = torch.utils.data.DataLoader(dataset = train_dataset, batch_size = 64, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = 1000, shuffle = False)

In [6]:
# Neural Network Model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 128)    # Input layer to hidden layer
        self.fc2 = nn.Linear(128, 10)       # Hidden layer to output layer

    def forward(self, x):
        x = x.view(-1, 28*28)   # Flatten the input
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = Net()

In [7]:
# Loss Function & Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum = 0.9)

In [8]:
# Model Training
for epoch in range(1, 6):   # 5 epochs
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()           # Zero the gradients
        output = model(data)            # Forward pass
        loss = criterion(output, target)# Compute Loss
        loss.backward()                 # Backward Pass
        optimizer.step()                # Update weights

        if batch_idx % 100 == 0:
            print(f'Epoch {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)}] Loss: {loss.item():.6f}')

Epoch 1 [0/60000] Loss: 2.289609
Epoch 1 [6400/60000] Loss: 0.372935
Epoch 1 [12800/60000] Loss: 0.319256
Epoch 1 [19200/60000] Loss: 0.342861
Epoch 1 [25600/60000] Loss: 0.098823
Epoch 1 [32000/60000] Loss: 0.127149
Epoch 1 [38400/60000] Loss: 0.225669
Epoch 1 [44800/60000] Loss: 0.320658
Epoch 1 [51200/60000] Loss: 0.120148
Epoch 1 [57600/60000] Loss: 0.082395
Epoch 2 [0/60000] Loss: 0.051805
Epoch 2 [6400/60000] Loss: 0.139695
Epoch 2 [12800/60000] Loss: 0.124839
Epoch 2 [19200/60000] Loss: 0.217064
Epoch 2 [25600/60000] Loss: 0.136984
Epoch 2 [32000/60000] Loss: 0.085872
Epoch 2 [38400/60000] Loss: 0.085886
Epoch 2 [44800/60000] Loss: 0.117071
Epoch 2 [51200/60000] Loss: 0.058490
Epoch 2 [57600/60000] Loss: 0.086237
Epoch 3 [0/60000] Loss: 0.099329
Epoch 3 [6400/60000] Loss: 0.075468
Epoch 3 [12800/60000] Loss: 0.074891
Epoch 3 [19200/60000] Loss: 0.083175
Epoch 3 [25600/60000] Loss: 0.047057
Epoch 3 [32000/60000] Loss: 0.041969
Epoch 3 [38400/60000] Loss: 0.043347
Epoch 3 [44800/6

In [9]:
# Evaluate
model.eval()
correct = 0
with torch.no_grad():
    for data, target in test_loader:
        output = model(data)
        pred = output.argmax(dim = 1, keepdim = True)   # Get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

print(f'Test Accuracy: {100. * correct / len(test_loader.dataset):.2f}%')

Test Accuracy: 97.69%


In [11]:
import tkinter as tk
from PIL import Image, ImageDraw, ImageOps # type: ignore
import torch # type: ignore
import cv2 # type: ignore
import numpy as np

# Load your trained model
model.eval()

# Create a Tkinter window
class App(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Draw a Digit")
        self.canvas = tk.Canvas(self, width=280, height=280, bg='white')
        self.canvas.pack()
        self.button = tk.Button(self, text="Predict", command=self.predict)
        self.button.pack()
        self.clear_button = tk.Button(self, text="Clear", command=self.clear)
        self.clear_button.pack()

        self.canvas.bind("<B1-Motion>", self.paint)
        self.image1 = Image.new("L", (280, 280), color=255)
        self.draw = ImageDraw.Draw(self.image1)

    def paint(self, event):
        x1, y1 = (event.x - 8), (event.y - 8)
        x2, y2 = (event.x + 8), (event.y + 8)
        self.canvas.create_oval(x1, y1, x2, y2, fill='black')
        self.draw.ellipse([x1, y1, x2, y2], fill=0)

    def clear(self):
        self.canvas.delete("all")
        self.draw.rectangle([0, 0, 280, 280], fill=255)

    def predict(self):
        # Resize and normalize the image to 28x28
        img = self.image1.resize((28, 28))
        img = ImageOps.invert(img)
        img = np.array(img).astype(np.float32) / 255.0
        img = torch.tensor(img).unsqueeze(0).unsqueeze(0)  # shape (1, 1, 28, 28)

        # Predict
        with torch.no_grad():
            output = model(img)
            pred = output.argmax(dim=1).item()
            print(f"Predicted digit: {pred}")
            self.title(f"Predicted: {pred}")

# Run the app
app = App()
app.mainloop()

Predicted digit: 9
Predicted digit: 9
Predicted digit: 7
Predicted digit: 2
Predicted digit: 8
Predicted digit: 8
Predicted digit: 6
Predicted digit: 6
Predicted digit: 5
Predicted digit: 5
Predicted digit: 5
Predicted digit: 5
Predicted digit: 5
Predicted digit: 3
Predicted digit: 3
Predicted digit: 3
Predicted digit: 3
Predicted digit: 3
Predicted digit: 3
Predicted digit: 3
Predicted digit: 3
Predicted digit: 3
Predicted digit: 8
Predicted digit: 8
Predicted digit: 8
Predicted digit: 8
Predicted digit: 8
Predicted digit: 3
Predicted digit: 3
Predicted digit: 4
Predicted digit: 3
Predicted digit: 2
Predicted digit: 2
Predicted digit: 2
Predicted digit: 2
Predicted digit: 2
Predicted digit: 2
Predicted digit: 3
Predicted digit: 3
Predicted digit: 3
Predicted digit: 3
Predicted digit: 3
Predicted digit: 3
Predicted digit: 3
Predicted digit: 3
Predicted digit: 5
Predicted digit: 3
Predicted digit: 3
Predicted digit: 3
Predicted digit: 3
